# PCA from decimated data — **global** PCA (recommended)

**Why global PCA:** A per-sequence or per–time-index PCA gives different coordinate systems per sample (PC1 in shot A ≠ PC1 in shot B). That breaks learnability: the NN sees incompatible features. A **global** PCA gives a consistent embedding: every time point in every sequence is projected onto the same axes.

**Pipeline:**
- **SNR > 3 (paper):** Optionally add `snr_min` to meta (from shot lists or computed from decimated data). Fit PCA and write output **only for shots with SNR > 3**.
- Split train/val/test at the **sequence (shot) level**.
- Fit PCA **only on training time points** (and only on training shots with SNR > 3): stream through those shots, accumulate mean μ ∈ ℝ¹⁶⁰ and covariance C ∈ ℝ¹⁶⁰×¹⁶⁰ with a single-pass online algorithm (Welford). Memory stays O(160²). Eigendecompose C and take top K eigenvectors W_K.
- Transform and save **only SNR > 3 shots**: z_t = (x_t − μ) W_K. Output dirs get `meta.csv` with `snr_min` column so downstream training (e.g. `run_fusion_soen --snr-min-threshold 3.0`) can use the same filter.

In [ ]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.decomposition import PCA
from tqdm import tqdm

# Paths — decimated inputs (disruptive + clear)
DECIMATED_ROOT = Path('/home/idies/workspace/Storage/yhuang2/persistent/ecei/dsrpt_decimated')
CLEAR_DECIMATED_ROOT = Path('/home/idies/workspace/Storage/yhuang2/persistent/ecei/clear_decimated')
# Outputs: N_PCA_OUT in {1, 4, 8, 16} → dirs dsrpt_decimated_pca{N}, clear_decimated_pca{N}
N_PCA_OUT = 1   # number of PCs to write (training default is 1)
OUT_DSRPT_PCA = DECIMATED_ROOT.parent / f'dsrpt_decimated_pca{N_PCA_OUT}'
OUT_CLEAR_PCA = DECIMATED_ROOT.parent / f'clear_decimated_pca{N_PCA_OUT}'

N_COMPONENTS = 16   # fit top K (global PCA); must be >= N_PCA_OUT
CHANNELS = 20 * 8   # 160

# Paper (Churchill et al.): "good ECEi data (SNR > 3)" — fit PCA and write output only for these shots
SNR_MIN_THRESHOLD = 3.0
# Optional: DisruptCNN-format shot lists (columns: Shot, ..., SNR min at index 5). If None, SNR is computed from decimated H5.
SHOT_LIST_DSRPT = None   # e.g. Path('disruptcnn/shots/d3d_disrupt_ecei.final.txt')
SHOT_LIST_CLEAR = None   # e.g. Path('disruptcnn/shots/d3d_clear_ecei.final.txt')

## 0. SNR > 3 (paper): add snr_min to meta, use only for PCA fit and output

Load SNR from DisruptCNN-format shot list files (column "SNR min") if `SHOT_LIST_DSRPT` / `SHOT_LIST_CLEAR` are set; otherwise compute a simple per-shot proxy from decimated data (min over channels of std(rest)/std(baseline)). **The computed proxy is not the same scale as the paper's curator SNR** — use shot lists when possible. Shots with **missing** snr_min (NaN) are **kept**; only shots with snr_min **known and ≤ threshold** are dropped.

In [ ]:
def _get_shot_lengths(root: Path, meta_path: Path = None) -> dict[int, int]:
    """Get {shot: T} from meta (n_samples/T column) if present, else from H5 shape without loading data. Uses parallel file opens for speed."""
    if meta_path and meta_path.exists():
        meta = pd.read_csv(meta_path)
        for col in ('n_samples', 'T', 'length', 'n_samples_decimated'):
            if col in meta.columns and 'shot' in meta.columns:
                return dict(zip(meta['shot'].astype(int), meta[col].astype(int)))
    def _one(p):
        if not p.stem.isdigit():
            return None
        try:
            with h5py.File(p, 'r') as f:
                return (int(p.stem), f['LFS'].shape[-1])
        except Exception:
            return None
    paths = [p for p in root.glob('*.h5') if p.stem.isdigit()]
    out = {}
    with ThreadPoolExecutor(max_workers=min(32, len(paths) or 1)) as ex:
        for fut in as_completed(ex.submit(_one, p) for p in paths):
            res = fut.result()
            if res is not None:
                out[res[0]] = res[1]
    return out
dsrpt_lengths = _get_shot_lengths(DECIMATED_ROOT, DECIMATED_ROOT / 'meta.csv') if DECIMATED_ROOT.exists() else {}
clear_lengths = _get_shot_lengths(CLEAR_DECIMATED_ROOT, CLEAR_DECIMATED_ROOT / 'meta.csv') if CLEAR_DECIMATED_ROOT.exists() else {}

def load_snr_from_shot_list(path: Path) -> dict[int, float]:
    """Load shot -> SNR min from DisruptCNN-format shot list (tab-sep, SNR min at column index 5)."""
    if path is None or not Path(path).exists():
        return {}
    data = np.loadtxt(path, skiprows=1)
    if data.size == 0:
        return {}
    if data.ndim == 1:
        data = data.reshape(1, -1)
    shots = data[:, 0].astype(int)
    snr = data[:, 5].astype(float)
    return dict(zip(shots, snr))

def compute_snr_from_decimated(root: Path, shot: int, T: int, baseline_frac: float = 0.1) -> float:
    """Simple SNR proxy: min over channels of (std(rest) / std(baseline)). Baseline = first baseline_frac of time."""
    with h5py.File(root / f'{shot}.h5', 'r') as f:
        X = np.asarray(f['LFS'][:], dtype=np.float64)  # (20, 8, T)
    X_flat = X.reshape(CHANNELS, -1).T  # (T, 160)
    n_base = max(1, int(T * baseline_frac))
    baseline = X_flat[:n_base]
    rest = X_flat[n_base:]
    if rest.shape[0] < 2:
        return np.nan
    std_base = np.std(baseline, axis=0)
    std_rest = np.std(rest, axis=0)
    std_base = np.where(std_base > 1e-12, std_base, 1e-12)
    ratio = std_rest / std_base
    return float(np.min(ratio))

# Load or compute snr_min per shot
snr_d = load_snr_from_shot_list(SHOT_LIST_DSRPT)
snr_c = load_snr_from_shot_list(SHOT_LIST_CLEAR)
if not snr_d and DECIMATED_ROOT.exists():
    print('Computing SNR from decimated disruptive data...')
    for shot in tqdm(list(dsrpt_lengths.keys())):
        snr_d[shot] = compute_snr_from_decimated(DECIMATED_ROOT, shot, dsrpt_lengths[shot])
if not snr_c and CLEAR_DECIMATED_ROOT.exists():
    print('Computing SNR from decimated clear data...')
    for shot in tqdm(list(clear_lengths.keys())):
        snr_c[shot] = compute_snr_from_decimated(CLEAR_DECIMATED_ROOT, shot, clear_lengths[shot])

# Load meta and add snr_min (so downstream and output meta have it)
meta_d = pd.read_csv(DECIMATED_ROOT / 'meta.csv') if (DECIMATED_ROOT / 'meta.csv').exists() else pd.DataFrame(columns=['shot', 'split', 't_disruption'])
meta_c = pd.read_csv(CLEAR_DECIMATED_ROOT / 'meta.csv') if CLEAR_DECIMATED_ROOT.exists() and (CLEAR_DECIMATED_ROOT / 'meta.csv').exists() else pd.DataFrame(columns=['shot', 'split'])
if 'shot' in meta_d.columns:
    meta_d['snr_min'] = meta_d['shot'].astype(int).map(snr_d)
if 'shot' in meta_c.columns:
    meta_c['snr_min'] = meta_c['shot'].astype(int).map(snr_c)

# Shots that pass SNR > threshold. Only drop when snr_min is known and <= threshold; keep when NaN (missing) to avoid over-dropping.
def _good_shots(meta, snr_col='snr_min'):
    if snr_col not in meta.columns or meta.empty:
        return set()
    s = pd.to_numeric(meta[snr_col], errors='coerce')
    # Keep: (snr_min > threshold) OR (snr_min is NaN = no info, keep to be safe)
    keep = (s > SNR_MIN_THRESHOLD) | s.isna()
    return set(meta.loc[keep, 'shot'].astype(int))

good_shots_d = _good_shots(meta_d) if not meta_d.empty else set(dsrpt_lengths.keys())
good_shots_c = _good_shots(meta_c) if not meta_c.empty else set(clear_lengths.keys())
if meta_d.empty and dsrpt_lengths:
    good_shots_d = set(dsrpt_lengths.keys())
if meta_c.empty and clear_lengths:
    good_shots_c = set(clear_lengths.keys())

# Diagnostics: so you can check if computed SNR or NaN is causing big drops
if 'snr_min' in meta_d.columns and not meta_d.empty:
    s = pd.to_numeric(meta_d['snr_min'], errors='coerce')
    n_na = s.isna().sum()
    s_ok = s.dropna()
    print(f'Disruptive snr_min: n={len(s)}  NaN={n_na}  valid: min={s_ok.min():.3f} median={s_ok.median():.3f} max={s_ok.max():.3f}  n>{SNR_MIN_THRESHOLD}={(s_ok > SNR_MIN_THRESHOLD).sum()}')
if 'snr_min' in meta_c.columns and not meta_c.empty:
    s = pd.to_numeric(meta_c['snr_min'], errors='coerce')
    n_na = s.isna().sum()
    s_ok = s.dropna()
    print(f'Clear snr_min:       n={len(s)}  NaN={n_na}  valid: min={s_ok.min():.3f} median={s_ok.median():.3f} max={s_ok.max():.3f}  n>{SNR_MIN_THRESHOLD}={(s_ok > SNR_MIN_THRESHOLD).sum()}')

# Save meta with snr_min back to decimated dirs (so run_fusion_soen --snr-min-threshold can use it when reading from _pca dirs)
if not meta_d.empty and (DECIMATED_ROOT / 'meta.csv').exists():
    meta_d.to_csv(DECIMATED_ROOT / 'meta.csv', index=False)
if not meta_c.empty and CLEAR_DECIMATED_ROOT.exists() and (CLEAR_DECIMATED_ROOT / 'meta.csv').exists():
    meta_c.to_csv(CLEAR_DECIMATED_ROOT / 'meta.csv', index=False)

n_drop_d = len(dsrpt_lengths) - len(good_shots_d) if dsrpt_lengths else 0
n_drop_c = len(clear_lengths) - len(good_shots_c) if clear_lengths else 0
print(f'SNR > {SNR_MIN_THRESHOLD} (or NaN): disruptive {len(good_shots_d)} kept, {n_drop_d} dropped; clear {len(good_shots_c)} kept, {n_drop_c} dropped')

## 1. List shots, lengths, and splits (train/val/test)

In [ ]:
def get_shot_lengths(root: Path) -> dict[int, int]:
    """Return {shot_id: T} for each .h5 in root."""
    out = {}
    for p in root.glob('*.h5'):
        if not p.stem.isdigit():
            continue
        shot = int(p.stem)
        with h5py.File(p, 'r') as f:
            T = f['LFS'].shape[-1]
        out[shot] = T
    return out

def get_splits(root: Path, lengths: dict) -> dict[int, str]:
    """Return {shot_id: 'train'|'val'|'test'} from meta.csv or 80/20 default."""
    if (root / 'meta.csv').exists():
        meta = pd.read_csv(root / 'meta.csv')
        if 'split' in meta.columns:
            return dict(zip(meta['shot'].astype(int), meta['split'].astype(str)))
    # Default: 80% train, 20% test
    shots = list(lengths.keys())
    n = len(shots)
    n_train = int(0.8 * n)
    out = {s: 'train' for s in shots[:n_train]}
    for s in shots[n_train:]:
        out[s] = 'test'
    return out

dsrpt_lengths = get_shot_lengths(DECIMATED_ROOT) if DECIMATED_ROOT.exists() else {}
clear_lengths = get_shot_lengths(CLEAR_DECIMATED_ROOT) if CLEAR_DECIMATED_ROOT.exists() else {}
dsrpt_splits = get_splits(DECIMATED_ROOT, dsrpt_lengths) if dsrpt_lengths else {}
clear_splits = get_splits(CLEAR_DECIMATED_ROOT, clear_lengths) if clear_lengths else {}

train_shots_d = [s for s, sp in dsrpt_splits.items() if sp == 'train']
train_shots_c = [s for s, sp in clear_splits.items() if sp == 'train']
# Restrict to SNR > threshold (good_shots from section 0) for PCA fit
train_shots_d = [s for s in train_shots_d if s in good_shots_d]
train_shots_c = [s for s in train_shots_c if s in good_shots_c]
print(f'Disruptive: {len(dsrpt_lengths)} shots  (train with SNR>3={len(train_shots_d)})')
print(f'Clear:      {len(clear_lengths)} shots  (train with SNR>3={len(train_shots_c)})')

## 2. Streaming mean and covariance (training time points only)

In [ ]:
# Welford in batches: merge (n, mean, M2) with batch (B, 160). Memory O(160²) + O(B*160).
BATCH_SIZE = 10_000   # time points per batch (vectorized update)

def welford_merge(n, mean, M2, batch: np.ndarray):
    """Merge batch (B, 160) into running stats. Returns (n_new, mean_new, M2_new)."""
    B = len(batch)
    if B == 0:
        return n, mean, M2
    batch_mean = batch.mean(axis=0)
    batch_M2 = (batch - batch_mean).T @ (batch - batch_mean)
    n_new = n + B
    mean_new = (n * mean + B * batch_mean) / n_new
    delta = mean - batch_mean
    M2_new = M2 + batch_M2 + (n * B / n_new) * np.outer(delta, delta)
    return n_new, mean_new, M2_new

def load_shot_flat(root: Path, shot: int, T: int) -> np.ndarray:
    """Load one shot as (T, 160) float64."""
    with h5py.File(root / f'{shot}.h5', 'r') as f:
        data = np.asarray(f['LFS'][:], dtype=np.float64)  # (20, 8, T)
    return data.reshape(CHANNELS, -1).T  # (T, 160)

def transform_shot_global(root: Path, shot: int, T: int, mu: np.ndarray, W_K: np.ndarray) -> np.ndarray:
    """z_t = (x_t - μ) W_K. Return (K, T) for LFS layout."""
    X = load_shot_flat(root, shot, T)  # (T, 160)
    Z = (X - mu) @ W_K  # (T, K)
    return Z.T.astype(np.float32)  # (K, T)

# Training shots only: (root, shot, T)
train_tuples = [(DECIMATED_ROOT, s, dsrpt_lengths[s]) for s in train_shots_d]
train_tuples += [(CLEAR_DECIMATED_ROOT, s, clear_lengths[s]) for s in train_shots_c]

n_total = 0
mean = np.zeros(CHANNELS, dtype=np.float64)
M2 = np.zeros((CHANNELS, CHANNELS), dtype=np.float64)

for root, shot, T in tqdm(train_tuples, desc='Streaming μ and M2 (train only)'):
    X = load_shot_flat(root, shot, T)  # (T, 160)
    for start in range(0, len(X), BATCH_SIZE):
        batch = X[start:start + BATCH_SIZE]
        n_total, mean, M2 = welford_merge(n_total, mean, M2, batch)

print(f'Training time points: {n_total}')
if n_total < 2:
    raise ValueError('Need at least 2 training time points to fit PCA')

## 3. Eigendecompose C and take top K eigenvectors

In [ ]:
C = M2 / (n_total - 1)
eigenvalues, eigenvectors = np.linalg.eigh(C)
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

K = min(N_COMPONENTS, CHANNELS, len(eigenvalues))
mu = mean.copy()
W_K = eigenvectors[:, :K].astype(np.float32)  # (160, K)

var_explained = eigenvalues / eigenvalues.sum()
print(f'Global PCA: μ ∈ R^{CHANNELS}, W_K ∈ R^{CHANNELS}×{K}')
print(f'Cumulative variance (top {K}): {var_explained[:K].sum():.4f}')

## 3b. How many components for ≥99% variance? (from global C)

In [ ]:
target_variance = 0.99
cumvar = np.cumsum(var_explained)
k_99 = int(np.searchsorted(cumvar, target_variance)) + 1
k_99 = min(k_99, len(cumvar))
print(f'Components needed for ≥{target_variance*100:.0f}% variance (global PCA): {k_99}')
print(f'  → Consider N_COMPONENTS >= {k_99} to retain ≥99% of the information.')

## 3c. Save PCA result (top 16 PCs; write 1/4/8/16 via N_PCA_OUT in 3d)

In [ ]:
N_PC_SAVE = 16   # save top 16 so we can write _pca1, _pca4, _pca8, _pca16
W_full = eigenvectors[:, :N_PC_SAVE].astype(np.float32)  # (160, 16)
pca_save_path = Path('pca_global_top16.npz')
np.savez(pca_save_path, mu=mu.astype(np.float32), W_K=W_full, var_explained=var_explained[:N_PC_SAVE].astype(np.float32))

info_saved = var_explained[:N_PC_SAVE].sum()
print(f'Saved top {N_PC_SAVE} PCs to {pca_save_path}')
print(f'Variance (information) retained: {info_saved*100:.2f}%')
print(f'Per-component: {var_explained[:N_PC_SAVE]}')

## 3d. Save transformed data (top N_PCA_OUT PCs → *_pca1, *_pca4, *_pca8, *_pca16)

In [ ]:
W_save = W_full[:, :N_PCA_OUT]
import shutil

# Only write shots with SNR > threshold; write meta with snr_min column
OUT_DSRPT_PCA.mkdir(parents=True, exist_ok=True)
meta_d_out = meta_d[meta_d['shot'].astype(int).isin(good_shots_d)] if not meta_d.empty else pd.DataFrame()
if meta_d_out.empty and good_shots_d:
    meta_d_out = pd.DataFrame({'shot': list(good_shots_d), 'snr_min': [snr_d.get(s, np.nan) for s in good_shots_d], 'split': [dsrpt_splits.get(s, 'train') for s in good_shots_d]})
    if not meta_d.empty and 't_disruption' in meta_d.columns:
        meta_d_out['t_disruption'] = meta_d_out['shot'].map(meta_d.set_index('shot')['t_disruption'])
if not meta_d_out.empty:
    meta_d_out.to_csv(OUT_DSRPT_PCA / 'meta.csv', index=False)
for shot in tqdm([s for s in dsrpt_lengths if s in good_shots_d], desc=f'Save disruptive {N_PCA_OUT}-PC (SNR>3)'):
    T = dsrpt_lengths[shot]
    data = transform_shot_global(DECIMATED_ROOT, shot, T, mu, W_save)
    with h5py.File(OUT_DSRPT_PCA / f'{shot}.h5', 'w') as f:
        f.create_dataset('LFS', data=data, dtype=np.float32)
print(f'Saved {len(good_shots_d)} disruptive shots (SNR>3) to {OUT_DSRPT_PCA} (shape {N_PCA_OUT}×T per shot)')

OUT_CLEAR_PCA.mkdir(parents=True, exist_ok=True)
meta_c_out = meta_c[meta_c['shot'].astype(int).isin(good_shots_c)] if not meta_c.empty else pd.DataFrame()
if meta_c_out.empty and good_shots_c:
    meta_c_out = pd.DataFrame({'shot': list(good_shots_c), 'snr_min': [snr_c.get(s, np.nan) for s in good_shots_c], 'split': [clear_splits.get(s, 'train') for s in good_shots_c]})
if not meta_c_out.empty:
    meta_c_out.to_csv(OUT_CLEAR_PCA / 'meta.csv', index=False)
for shot in tqdm([s for s in clear_lengths if s in good_shots_c], desc=f'Save clear {N_PCA_OUT}-PC (SNR>3)'):
    T = clear_lengths[shot]
    data = transform_shot_global(CLEAR_DECIMATED_ROOT, shot, T, mu, W_save)
    with h5py.File(OUT_CLEAR_PCA / f'{shot}.h5', 'w') as f:
        f.create_dataset('LFS', data=data, dtype=np.float32)
print(f'Saved {len(good_shots_c)} clear shots (SNR>3) to {OUT_CLEAR_PCA} (shape {N_PCA_OUT}×T per shot)')

## 4. Transform and save disruptive shots -> dsrpt_decimated_pca

In [ ]:
OUT_DSRPT_PCA.mkdir(parents=True, exist_ok=True)
meta_d_out = meta_d[meta_d['shot'].astype(int).isin(good_shots_d)] if not meta_d.empty else meta_d
if not meta_d_out.empty:
    meta_d_out.to_csv(OUT_DSRPT_PCA / 'meta.csv', index=False)
for shot in tqdm([s for s in dsrpt_lengths if s in good_shots_d], desc='Save disruptive PCA (SNR>3)'):
    data = transform_shot_global(DECIMATED_ROOT, shot, dsrpt_lengths[shot], mu, W_K)
    with h5py.File(OUT_DSRPT_PCA / f'{shot}.h5', 'w') as f:
        f.create_dataset('LFS', data=data, dtype=np.float32)
print(f'Saved {len(good_shots_d)} disruptive shots (SNR>3) to {OUT_DSRPT_PCA}')

## 5. Transform and save clear shots -> clear_decimated_pca

In [ ]:
OUT_CLEAR_PCA.mkdir(parents=True, exist_ok=True)
meta_c_out = meta_c[meta_c['shot'].astype(int).isin(good_shots_c)] if not meta_c.empty else meta_c
if not meta_c_out.empty:
    meta_c_out.to_csv(OUT_CLEAR_PCA / 'meta.csv', index=False)
for shot in tqdm([s for s in clear_lengths if s in good_shots_c], desc='Save clear PCA (SNR>3)'):
    data = transform_shot_global(CLEAR_DECIMATED_ROOT, shot, clear_lengths[shot], mu, W_K)
    with h5py.File(OUT_CLEAR_PCA / f'{shot}.h5', 'w') as f:
        f.create_dataset('LFS', data=data, dtype=np.float32)
print(f'Saved {len(good_shots_c)} clear shots (SNR>3) to {OUT_CLEAR_PCA}')

## 6. Sanity: shapes

In [ ]:
if good_shots_d:
    shot0 = next(iter(good_shots_d))
    with h5py.File(OUT_DSRPT_PCA / f'{shot0}.h5', 'r') as f:
        sh = f['LFS'].shape
    print(f'Disruptive PCA example: shot {shot0} LFS shape = {sh} (N_components, T)')
if good_shots_c:
    shot0 = next(iter(good_shots_c))
    with h5py.File(OUT_CLEAR_PCA / f'{shot0}.h5', 'r') as f:
        sh = f['LFS'].shape
    print(f'Clear PCA example:      shot {shot0} LFS shape = {sh} (N_components, T)')